In [3]:
import numpy as np
import tensorflow as tf
sess = tf.Session()

In [4]:
#We want to create a network that has only one LSTM cell. 
#We have to pass 2 elements to LSTM, the prv_output and prv_state, so called, h and c. 
#Therefore, we initialize a state vector, state. 
#Here, state is a tuple with 2 elements, each one is of size [1 x 4], one for passing prv_output 
#to next time step, and another for passing the prv_state to next time stamp.

In [5]:
#variables
LSTM_CELL_SIZE = 4  # output size (dimension), which is same as hidden size in the cell
lstm_cell = tf.contrib.rnn.BasicLSTMCell(LSTM_CELL_SIZE, state_is_tuple=True)
state = (tf.zeros([1,LSTM_CELL_SIZE]),)*2
state


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.


(<tf.Tensor 'zeros:0' shape=(1, 4) dtype=float32>,
 <tf.Tensor 'zeros:0' shape=(1, 4) dtype=float32>)

In [6]:
#Let define a sample input. In this example, batch_size = 1, and seq_len = 6:

sample_input = tf.constant([[3,2,2,2,2,2]],dtype=tf.float32)
print (sess.run(sample_input))

[[3. 2. 2. 2. 2. 2.]]


In [7]:
#Now, we can pass the input to lstm_cell, and check the new state:
with tf.variable_scope("LSTM_sample1"):
    output, state_new = lstm_cell(sample_input, state)
sess.run(tf.global_variables_initializer())
print (sess.run(state_new))

Instructions for updating:
Colocations handled automatically by placer.
LSTMStateTuple(c=array([[-0.14780639, -0.408931  ,  0.50122803,  0.5403549 ]],
      dtype=float32), h=array([[-0.10666159, -0.1948254 ,  0.2981056 ,  0.04786894]],
      dtype=float32))


In [8]:
#As we can see, the states has 2 parts, the new state c, and also the output h. Lets check the output again:
print (sess.run(output))

[[-0.10666159 -0.1948254   0.2981056   0.04786894]]


## Stacked LSTM
#What about if we want to have a RNN with stacked LSTM? For example, a 2-layer LSTM. 
#In this case, the output of the first layer will become the input of the second.
#Lets start with a new session:

In [13]:
sess = tf.Session()
input_dim = 6

#Lets create the stacked LSTM cell:
cells = []

#Creating the first layer LTSM cell.

LSTM_CELL_SIZE_1 = 4 #4 hidden nodes
cell1 = tf.keras.layers.LSTMCell (LSTM_CELL_SIZE_1)
cells.append(cell1)

In [14]:
#Creating the second layer LTSM cell.
LSTM_CELL_SIZE_2 = 5 #5 hidden nodes
cell2 = tf.keras.layers.LSTMCell (LSTM_CELL_SIZE_2)
cells.append(cell2)

In [15]:
#To create a multi-layer LTSM we use the tf.contrib.rnnMultiRNNCell function, 
#it takes in multiple single layer LTSM cells to create a multilayer stacked LTSM model.
#stacked_lstm = tf.contrib.rnn.MultiRNNCell(cells)
stacked_lstm = tf.keras.layers.StackedRNNCells(cells)

In [16]:
#Now we can create the RNN from stacked_lstm:
# Batch size x time steps x features.
data = tf.placeholder(tf.float32, [None, None, input_dim])
output, state = tf.nn.dynamic_rnn(stacked_lstm, data, dtype=tf.float32)
#output, state = tf.keras.layers.RNN(stacked_lstm, data, dtype=tf.float32)

In [18]:
#Lets say the input sequence length is 3, and the dimensionality of the inputs is 6. 
#The input should be a Tensor of shape: [batch_size, max_time, dimension], in our case it would be (2, 3, 6)

#Batch size x time steps x features.
sample_input = [[[1,2,3,4,3,2], [1,2,1,1,1,2],[1,2,2,2,2,2]],[[1,2,3,4,3,2],[3,2,2,1,1,2],[0,0,0,0,3,2]]]
sample_input

[[[1, 2, 3, 4, 3, 2], [1, 2, 1, 1, 1, 2], [1, 2, 2, 2, 2, 2]],
 [[1, 2, 3, 4, 3, 2], [3, 2, 2, 1, 1, 2], [0, 0, 0, 0, 3, 2]]]

In [22]:
#we can now send our input to network, and check the output:
sess.run(tf.global_variables_initializer())
sess.run(output, feed_dict={data: sample_input})
output

InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype float and shape [?,?,6]
	 [[node Placeholder_2 (defined at <ipython-input-17-5c9901f36ee9>:3) ]]

Caused by op 'Placeholder_2', defined at:
  File "C:\Users\Yves\AppData\Local\Programs\Python\Python37\Lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Yves\AppData\Local\Programs\Python\Python37\Lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Yves\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Yves\AppData\Local\Programs\Python\Python37\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Yves\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\Yves\AppData\Local\Programs\